# Voice Gender Recognition (Deep Learning)

*Import libraries*

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from modules.mlp import MultilayerPerceptron
from modules.utils import encode_labels
from tqdm import tqdm

*Global parameters*

In [2]:
num_features = 20

*Paths*

In [3]:
data_path = '../data/'

## 1. Preprocessing

*Read dataset*

In [4]:
if num_features == 20:
    dataset = pd.read_csv(data_path + 'voice.csv')
elif num_features == 2:
    dataset = pd.read_csv(data_path + 'embedded_voice.csv')
else:
    raise NotImplementedError('Unsupported scenario')

*Feature matrix*

In [5]:
features = dataset.iloc[:, :-1].values

*Label vector*

In [6]:
labels = dataset.iloc[:, -1].values

## 2. Learning

*Split in train and rest*

In [7]:
features_train, features_rest, labels_train, labels_rest = train_test_split(features, labels, test_size=0.40)

*Split in validation and test*

In [8]:
features_val, features_test, labels_val, labels_test = train_test_split(features, labels, test_size=0.50)

*Scale features*

In [9]:
sc = MinMaxScaler()
features_train = sc.fit_transform(features_train)
features_val = sc.transform(features_val)
features_test = sc.transform(features_test)

### Multilayer Perceptron

*Build architectures*

In [10]:
architectures = []

def build_architectures(architecture):
    if len(architecture) >= 1:
        architecture = architecture
        architectures.append(architecture)
    if len(architecture) < 3:
        for i in range(5, 10 + 1):
            build_architectures(architecture + [i])

build_architectures([])

*Compute performance metrics*

In [11]:
def compute_metrics(labels_eval, labels_pred, labels_scores):
    metrics = {}
    tn, fp, fn, tp = confusion_matrix(labels_eval, labels_pred).ravel()
    metrics['accuracy'] = (tp + tn) / (tp + tn + fp + fn)
    metrics['sensitivity'] = tp / (tp + fn)
    metrics['specificity'] = tn / (tn + fp)
    metrics['fpr'], metrics['tpr'], _ = roc_curve(labels_eval, labels_scores[:, 1])
    metrics['auc'] = auc(metrics['fpr'], metrics['tpr'])
    return metrics

*Assess classifier's performance*

In [12]:
def assess_performance(classifier, features_eval, labels_eval):
    labels_scores = classifier.predict(features_eval)
    labels_pred = np.argmax(labels_scores, axis=1)
    return compute_metrics(labels_eval, labels_pred, labels_scores)

*Build scenario id*

In [13]:
def build_scenario_id(learning_rate, architecture):
    scheme = '-'.join(str(layer) for layer in architecture)
    return '{:.2f}-{}'.format(learning_rate, scheme)

*Arrange gradients' mean norms*

In [14]:
def arrange_gradients_mean_norms(gradients_mean_norms, architecture):
    return {layer + 1 : gradients_mean_norms[:, layer] for layer in range(len(architecture) - 1)}

*Encode training labels*

In [15]:
encoded_labels_train = encode_labels(labels_train)

*Train models*

In [16]:
brains_val = {}
brains_test = {}
brains_errors = {}
brains_gradients = {}

# Sweep over architectures
for architecture in tqdm(architectures):
    # Select architecture
    architecture = [features.shape[1]] + architecture + [encoded_labels_train.shape[1]]
    # Sweep over learning rates
    for eta in [0.2, 0.5, 0.9]:
        # Scenario id
        scenario_id = build_scenario_id(eta, architecture)
        # Build brain
        brain = MultilayerPerceptron(architecture, learning_rate=eta)
        # Train brain
        network_avg_errors, gradients_mean_norms = brain.fit(features_train, encoded_labels_train)
        # Save training outcomes
        brains_errors[scenario_id] = network_avg_errors
        brains_gradients[scenario_id] = arrange_gradients_mean_norms(gradients_mean_norms, architecture)
        # Evaluate classifier's performance in the validation set
        metrics_val = assess_performance(brain, features_val, labels_val)
        brains_val[scenario_id] = metrics_val
        # Evaluate classifier's performance in the test set
        metrics_test = assess_performance(brain, features_test, labels_test)
        brains_test[scenario_id] = metrics_test

100%|████████████████████████████████████████████████████████████████████████████| 258/258 [12:32:35<00:00, 175.02s/it]


*Encase results*

In [17]:
brains_val = pd.DataFrame(brains_val).T
brains_test = pd.DataFrame(brains_test).T
brains_errors = pd.DataFrame(brains_errors).T
brains_gradients = pd.DataFrame(brains_gradients).T

*Persist results*

In [18]:
brains_val.to_csv(data_path + 'brains_val_{}d.csv'.format(num_features))
brains_test.to_csv(data_path + 'brains_test_{}d.csv'.format(num_features))
brains_errors.to_csv(data_path + 'brains_errors_{}d.csv'.format(num_features))
brains_gradients.to_csv(data_path + 'brains_gradients_{}d.csv'.format(num_features))